# Quantitative Momentum Strategy V2

Nessa segunda versão vou usar o retorno não apenas anual, como no último mês, no último trimestre, semestre e último ano.

In [1]:
import numpy as np
import pandas as pd
import math
import datetime
from scipy import stats
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [2]:
tickets = pd.read_csv('./data/BOVA11_holdings.csv')
tickets = tickets['Código']
tickets

0      VALE3
1      ITUB4
2      PETR4
3      BBDC4
4      B3SA3
       ...  
76     HGTX3
77     ECOR3
78    CVCB11
79      XBZ0
80          
Name: Código, Length: 81, dtype: object

In [6]:
columns = ['Ticker', 'Price', 'Number of Shares to Buy', 
           'One-Year Price Return', 'One-Year Return Percentile', 
           'Six-Month Price Return', 'Six-Month Return Percentile', 
           'Three-Month Price Return', 'Three-Month Return Percentile', 
           'One-Month Price Return', 'One-Month Return Percentile', 'HQM Score']
stocks = pd.DataFrame(columns = columns)
today = datetime.datetime.today().strftime('%Y-%m-%d')
last_year = datetime.datetime.now() - datetime.timedelta(days=365)
last_year = last_year.strftime('%Y-%m-%d')

for ticket in tickets:
    try:
        data = pdr.get_data_yahoo(ticket+'.SA', start=last_year, end=today)
        close = list(data['Close'])
        size = len(close)
        one_year_return = (close[size-1]-close[0])/close[0]
        six_month_return = (close[math.floor(size/2)-1]-close[0])/close[0]
        three_month_return = (close[math.floor(size/4)-1]-close[0])/close[0]
        one_month_return = (close[math.floor(size/12)-1]-close[0])/close[0]
        price = close[size-1]

        stocks = stocks.append(pd.Series([ticket, 
                                        price, 'N/A', 
                                        one_year_return, 'N/A', 
                                        six_month_return, 'N/A', 
                                        three_month_return, 'N/A', 
                                        one_month_return, 'N/A', 
                                        'N/A'], 
                                        index = columns), 
                                ignore_index = True)
    except:
        pass

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
stocks_copy = stocks.copy()

In [11]:
deltas = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for i in range(stocks_copy.shape[0]):
    for delta in deltas:
        col = '{} Return Percentile'.format(delta)
        _col = '{} Price Return'.format(delta)
        stocks_copy.loc[i, col] = stats.percentileofscore(stocks_copy[_col], stocks_copy.loc[i, _col])/100

In [15]:
from statistics import mean

for i in range(stocks_copy.shape[0]):
    momentuns = []
    for delta in deltas:
        col = '{} Return Percentile'.format(delta)
        momentuns.append(stocks_copy.loc[i, col])
    stocks_copy.loc[i, 'HQM Score'] = mean(momentuns)

In [16]:
stocks_copy.sort_values(by='HQM Score', ascending=False)
stocks_copy = stocks_copy[:25]

In [17]:
stocks_copy

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VALE3,82.269997,N/A,0.599339,0.909091,0.069013,0.818182,-0.208009,0.428571,0.036159,0.272727,0.607143
1,ITUB4,30.250000,N/A,-0.172367,0.350649,-0.239398,0.285714,-0.283995,0.246753,-0.044870,0.0649351,0.237013
2,PETR4,27.410000,N/A,-0.092685,0.441558,-0.281033,0.207792,-0.475339,0.0779221,0.006289,0.181818,0.227273
3,BBDC4,25.629999,N/A,-0.259249,0.181818,-0.336705,0.155844,-0.277746,0.25974,0.013584,0.207792,0.201299
4,B3SA3,58.529999,N/A,0.228332,0.766234,0.071773,0.831169,-0.107030,0.701299,-0.062959,0.038961,0.584416
5,PETR3,28.100000,N/A,-0.128141,0.402597,-0.295997,0.168831,-0.492088,0.0649351,0.000000,0.149351,0.196429
6,ABEV3,14.720000,N/A,-0.201735,0.285714,-0.223427,0.298701,-0.254338,0.298701,0.020607,0.220779,0.275974
7,MGLU3,23.719999,N/A,1.069357,0.974026,0.360524,0.961039,-0.060414,0.818182,0.135878,0.792208,0.886364
8,ITSA4,11.170000,N/A,-0.183480,0.324675,-0.242690,0.272727,-0.239766,0.376623,-0.017544,0.0909091,0.266234
9,WEGE3,71.970001,N/A,1.255406,1,0.435600,0.974026,0.313381,1,0.044187,0.337662,0.827922


In [20]:
portfolio_size = 1e8
position_size = portfolio_size/stocks_copy.shape[0]
for i in range(stocks_copy.shape[0]):
    stocks_copy.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/stocks_copy['Price'][i])

/home/ne0/.local/lib/python3.8/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [21]:
stocks_copy

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VALE3,82.269997,48620,0.599339,0.909091,0.069013,0.818182,-0.208009,0.428571,0.036159,0.272727,0.607143
1,ITUB4,30.250000,132231,-0.172367,0.350649,-0.239398,0.285714,-0.283995,0.246753,-0.044870,0.0649351,0.237013
2,PETR4,27.410000,145932,-0.092685,0.441558,-0.281033,0.207792,-0.475339,0.0779221,0.006289,0.181818,0.227273
3,BBDC4,25.629999,156067,-0.259249,0.181818,-0.336705,0.155844,-0.277746,0.25974,0.013584,0.207792,0.201299
4,B3SA3,58.529999,68341,0.228332,0.766234,0.071773,0.831169,-0.107030,0.701299,-0.062959,0.038961,0.584416
5,PETR3,28.100000,142348,-0.128141,0.402597,-0.295997,0.168831,-0.492088,0.0649351,0.000000,0.149351,0.196429
6,ABEV3,14.720000,271739,-0.201735,0.285714,-0.223427,0.298701,-0.254338,0.298701,0.020607,0.220779,0.275974
7,MGLU3,23.719999,168634,1.069357,0.974026,0.360524,0.961039,-0.060414,0.818182,0.135878,0.792208,0.886364
8,ITSA4,11.170000,358102,-0.183480,0.324675,-0.242690,0.272727,-0.239766,0.376623,-0.017544,0.0909091,0.266234
9,WEGE3,71.970001,55578,1.255406,1,0.435600,0.974026,0.313381,1,0.044187,0.337662,0.827922
